In [ ]:
pip install transformers torch
pip install transformers tensorflow


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 평가할 모델 목록
models = [
    "gpt2",
    "EleutherAI/gpt-neo-125M",
    "EleutherAI/gpt-j-6B",
    # 추가 모델이 있다면 여기에 추가
]

# 테스트할 프롬프트 목록
prompts = [
    "다음 영어 문장을 프랑스어로 번역하세요:\n\n'Artificial intelligence is transforming the world.'",
    "다음 기사에 대한 핵심 내용을 요약하세요:\n\n'기후 변화의 영향이 가속화되고 있다고 과학자들이 경고한다.'",
    "블록체인 기술의 개념을 쉬운 용어로 설명하세요.",
]

def generate_response(model_name, prompt):
    """주어진 프롬프트에 대한 모델의 응답을 생성합니다."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(
        inputs,
        max_length=inputs.shape[1] + 100,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_text = response[len(prompt):].strip()
    return generated_text

# 프롬프트와 모델을 순회하며 응답을 수집
for prompt in prompts:
    print(f"프롬프트:\n{prompt}\n{'='*60}")
    for model_name in models:
        try:
            response = generate_response(model_name, prompt)
            print(f"모델: {model_name}\n응답:\n{response}\n{'-'*60}")
        except Exception as e:
            print(f"모델: {model_name}\n에러 발생: {e}\n{'-'*60}")
    print("\n\n")



  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ikw2eys5
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ikw2eys5
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [ ]:
python prompt_transferability.py
